In [1]:
import xml.etree.ElementTree as ET
import os
import numpy as np
import subprocess
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
from random import randrange
from collections import defaultdict

In [2]:
bm_folder = '../benchmarks/'
file_name = 'sparcT1_core_stratixiv_arch_timing'
path_blif = os.path.join(bm_folder, 'blif',file_name +'.blif')

arch_name = 'stratixiv_arch.timing.xml'
path_arch_folder = os.path.join(bm_folder,'architecture' ,arch_name)

tree = ET.parse(path_arch_folder)
root = tree.getroot()

### net graph

In [3]:
class NetGraph:

        def __init__(self, vertices):
            # No. of vertices
            self.V = len(vertices)

            # default dictionary to store graph
            self.graph = defaultdict(list)

            self.Time = 0
            
            self.vertice_to_alias = dict(zip(list(vertices), np.arange(len(vertices))))
            self.alias_to_vertice = dict(zip(np.arange(len(vertices)), list(vertices)))
            assert self.vertice_to_alias[self.alias_to_vertice[100]] == 100
            
        # function to add an edge to graph
        def addEdge(self, u, v):
            self.graph[self.vertice_to_alias[u]].append(self.vertice_to_alias[v])
        
        
        # Prints all not yet visited vertices reachable from s
        def DFS(self, s):           # prints all vertices in DFS manner from a given source.
                                    # Initially mark all vertices as not visited
            visited = [False for i in range(self.V)]

            # Create a stack for DFS
            stack = []

            # Push the current source node.
            stack.append(s)
            
            # Descendants
            descendants = []
            
            while (len(stack)):
                # Pop a vertex from stack and print it
                s = stack[-1]
                stack.pop()

                # Stack may contain same vertex twice. So
                # we need to print the popped item only
                # if it is not visited.
                if (not visited[s]):
                    descendants.append(s)
                    visited[s] = True

                # Get all adjacent vertices of the popped vertex s
                # If a adjacent has not been visited, then push it
                # to the stack.
                for node in self.graph[s]:
                    if (not visited[node]):
                        stack.append(node)
            
            return descendants
        
        
#         def DFS(self, u, visited):
#             visited[u] = True
#             children = []
#             for v in self.graph[u]:
#                 if not visited[v]:
#                     children.append(v)
#                     children.extend(self.DFS(v, visited))   
#             print(children)
#             print(sum(visited))
#             return children
            
        def get_descendants(self, u):
            visited = [False] * (self.V)
            u = self.vertice_to_alias[u]
            descendants = self.DFS(u)
            return [self.alias_to_vertice[v] for v in descendants]
            

### load subckts + anotate datapath inputs 

In [4]:
file1 = open(path_blif, 'r')
lines = file1.readlines()

1) parse models (except for main)

In [5]:
# Strips the newline character
models = []
index1 = 0
for index2, line in enumerate(lines):
    if '.model' in line:
        models.append(lines[index1:index2])
        index1 = index2
models.append(lines[index1:])
main_model = models[1]
subckt_models = models[2:]

In [6]:
class Subckt:
    def __init__(self, name, inputs, outputs):
        self.data_inputs = []
        self.ctrl_inputs = []
        self.inputs = inputs
        self.outputs = outputs
        self.name = name

In [44]:
subckts = {}

for subckt_model in subckt_models:
    input_line, output_line = False, False
    inputs = []
    outputs = []
    for line in subckt_model:
        if '.model' in line:
            name = line.split()[1]
        if '.inputs' in line:
            input_line = True
            output_line = False
        elif '.outputs' in line:
            input_line = False
            output_line = True 
        elif line[0] == '.':
            input_line = False
            output_line = False
        elif input_line:
            inputs.append(line.split()[0])
        elif output_line:
            outputs.append(line.split()[0])
    
    subckts[name] = Subckt(name, inputs, outputs)
for subckt in subckts.values():
    for input_ in subckt.inputs:
        if 'data' in input_:
            subckt.data_inputs.append(input_)
        else:
            subckt.ctrl_inputs.append(input_)
            
subckts['dffeas'].ctrl_inputs.remove('d')
subckts['dffeas'].ctrl_inputs.remove('ena')
subckts['dffeas'].data_inputs.append('d')
subckts['dffeas'].data_inputs.append('ena')

In [8]:
new_main_model = []

new_line_split = []
for line in main_model:
    line_split = line.split()
    if line_split != [] and (line_split[-1] == '\\'):
        new_line_split.extend(line_split[:-1])
#     elif line_split != [] and (line_split[0]=='.names' or line_split[0] == '0' or line_split[0] == '1'):
#         new_line_split.extend(line_split)
    else:
        new_line_split.extend(line_split)
        new_line = ' '.join(new_line_split)
        new_main_model.append(new_line_split)
        new_line_split = []

In [9]:
# collect .inputs, .outputs, .names, .latch, .subckt
inputs_main = None
outputs_main = None
names_main = []
latches_main = []
subckts_main = []

for split_line in new_main_model:
    if split_line == []:
        pass
    elif split_line[0] == '.inputs':
        inputs_main = split_line
    elif split_line[0] == '.outputs':
        outputs_main = split_line
    elif split_line[0] == '.names': 
        names_main.append(split_line)
    elif split_line[0] == '.latch':
        latches_main.append(split_line)
    elif split_line[0] == '.subckt':
        subckts_main.append(split_line)

In [10]:
subckt_collection = [] #[[subckt_name, inputs, clk, outputs]]
circuit_atoms = []
counts = []
for subckt in subckts_main:
        #subckt first: '.subckt', second: 'name', then 'IO_name=netname'
        name = subckt[1]
        ins = {}
        clks = []
        outs = {}
        for io_net in subckt[2:]:
            port, net = io_net.split('=')
            if port in subckts[name].inputs:
                if 'clk' in port:
                    clks.append(port)
                else:
                    ins[port] = net
            elif port in subckts[name].outputs:
                outs[port]= net
        
        input_ports = list(ins)
        output_ports = list(outs)
        circuit_atoms.append([name, ins, clks, outs])
        if not [name, input_ports, clks, output_ports] in subckt_collection:
            subckt_collection.append([name, input_ports, clks, output_ports])
            counts.append(1)
        else:
            i = subckt_collection.index([name, input_ports, clks, output_ports])
            counts[i] += 1

### net graph

Collect all nets, map to natural numbers

In [11]:
nets = set()
for atom in circuit_atoms:
    _, ins, _, outs = atom
    for net in list(ins.values()) + list(outs.values()):
        nets.add(net)

Make graph, edges between output nets and input nets of data ports

In [12]:
data_net_graph = NetGraph(nets)

#Voeg toe in tegengestelde richting: output naar input --> nodig om vanaf uitgang te zoeken 
for atom in circuit_atoms:
    name, ins, _, outs = atom
    subckt = subckts[name]
    for in_port, in_net in ins.items():
        if in_port in subckt.data_inputs:
            for out_port, out_net in outs.items():
                data_net_graph.addEdge(out_net, in_net)

In [54]:
ctrl_net_ends = []

for atom in circuit_atoms:
    name, ins, _, outs = atom
    subckt = subckts[name]
    for in_port, in_net in ins.items():
        if in_port in subckt.ctrl_inputs:
            ctrl_net_ends.append(in_net)
            
ctrl_net_ends = list(set(ctrl_net_ends))

In [52]:
nets_source_sinks = defaultdict(lambda: defaultdict(list))

for atom in circuit_atoms:
    name, ins, _, outs = atom
    for port, net in ins.items():
        nets_source_sinks[net]['sinks'].append(port)
    for port, net in outs.items():
        nets_source_sinks[net]['source'] = port

'lsu:lsu|lsu_qctl2:qctl2|dfq_vld_entries[4]~19'

In [80]:
count = 0
for i, ctrl_net_end in enumerate(ctrl_net_ends):
    if not nets_source_sinks[ctrl_net_end]['source'] in ['shareout', 'cout']:
        print(i, nets_source_sinks[ctrl_net_end]['source'], ctrl_net_end)
        count += 1
count

10 combout sparc_ifu:ifu|bw_r_icd:icd|icdata_ary_11_10~0
12 q sparc_ifu:ifu|bw_r_icd:icd|wr_index0[6]
14 combout tlu:tlu|bw_r_rf16x160:mra|bw_r_rf16x2:arr10|always0~0
17 q lsu:lsu|bw_r_idct:dtag|index_y[3]
18 combout lsu:lsu|lsu_dctldp:dctldp|asi_d[6]~2
22 combout sparc_ifu:ifu|bw_r_icd:icd|icdata_ary_11_11.raddr_a[5]~8
24 combout sparc_ifu:ifu|sparc_ifu_errdp:errdp|dff_s:t1_eadr_reg|q[36]~12
26 combout lsu:lsu|bw_r_tlb:dtlb|bw_r_tlb_tag_ram:bw_r_tlb_tag_ram|wr_addr[3]~2
27 combout sparc_ifu:ifu|sparc_ifu_ifqctl:ifqctl|ifc_ifd_addr_sel_bist_i2_l
28 combout sparc_exu:exu|sparc_exu_byp:bypass|dff_s:dff_rd_data_e2m|q[38]~10
32 combout sparc_ifu:ifu|bw_r_icd:icd|icdata_ary_00_10~0
34 combout sparc_ifu:ifu|bw_r_tlb:itlb|bw_r_tlb_data_ram:bw_r_tlb_data_ram|wr_addr[3]~3
39 combout lsu:lsu|lsu_qctl2:qctl2|dfq_rptr[2]~0
44 combout lsu:lsu|bw_r_tlb:dtlb|bw_r_tlb_data_ram:bw_r_tlb_data_ram|wr_addr[3]~3
48 combout sparc_ifu:ifu|bw_r_icd:icd|icdata_ary_11_11.raddr_a[3]~16
49 q sparc_exu:exu|bw_r_ir

294

In [40]:
net_graph = NetGraph(nets)

#Voeg toe in tegengestelde richting: output naar input --> nodig om vanaf uitgang te zoeken 
for atom in circuit_atoms:
    name, ins, _, outs = atom
    for in_port, in_net in ins.items():
        for out_port, out_net in outs.items():
            net_graph.addEdge(out_net, in_net)

In [16]:
ctrl_nets = set()
for i, net in enumerate(ctrl_net_ends):
    ctrl_nets.update(net_graph.get_descendants(net))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [72]:
len(ctrl_nets)

91541

In [75]:
list(ctrl_net_ends)[90]

'tlu:tlu|tlu_incr64:tick_incr64|Add0~86'

In [74]:
len(net_graph.get_descendants(list(ctrl_net_ends)[90]))

91536

In [18]:
len(ctrl_nets), len(nets)

(91541, 92829)

### make net graph: datapath edges only

In [15]:
data_nets = set()
for output_ in outputs_main:
    if output_ in nets:
        data_nets.update(data_net_graph.get_descendants(output_))


In [22]:
len(data_nets) + len(ctrl_nets), len(nets) 

(92791, 92829)

In [73]:
net_graph.get_descendants(outputs_main[123])

['spc_pcx_data_pa[116]',
 'lsu:lsu|lsu_qdp1:qdp1|mux4ds:pcx_pkt_src|Mux7~0',
 'tlu:tlu|sparc_tlu_intdp:intdp|dp_mux4ds:int_dsp_mux|Mux0~0',
 'tlu:tlu|sparc_tlu_intdp:intdp|dff_s:t0_indr_reg|q[10]',
 'tlu:tlu|tlu_hyperv:tlu_hyperv|inc_ind_asi_wr_indr[0]~0',
 'tlu:tlu|tlu_hyperv:tlu_hyperv|dffr_s:dffr_asi_indr_rw_g|q[0]',
 'sparc_ffu:ffu|sparc_ffu_ctl:ctl|dffrl_async:rstff|q[0]',
 'cmp_arst_l',
 'bw_clk_cl_sparc_cmp:spc_hdr|cluster_header:I0|cluster_grst_l',
 'cmp_grst_l',
 'tlu:tlu|tlu_hyperv:tlu_hyperv|dffr_s:dffr_asi_indr_rw_m|q[0]',
 'tlu:tlu|tlu_hyperv:tlu_hyperv|dffr_s:dffr_asi_indr_rw_m|q~2',
 'tlu:tlu|tlu_tdp:tdp|mux2ds:mx_asi_state_final_e|dout[0]~1',
 'sparc_ifu:ifu|sparc_ifu_fdp:fdp|dff_s:rbinst_e_reg|q[6]',
 'sparc_ifu:ifu|sparc_ifu_fdp:fdp|dff_s:rbinst_d_reg|q[6]',
 'sparc_ifu:ifu|sparc_ifu_fcl:fcl|fcl_fdp_inst_sel_switch_s_l~2',
 'sparc_ifu:ifu|sparc_ifu_fcl:fcl|immu_fault_f~0',
 'sparc_ifu:ifu|sparc_ifu_fcl:fcl|tlbmiss_s1_crit~2',
 'sparc_ifu:ifu|sparc_ifu_fcl:fcl|dff_s:un

In [78]:
len(nets)

92829

### make net graph: controlpath edges only

### intersection of datapath edges and control edges, are they separated?

In [81]:
## conclusie: een aantal netten grootteorde 200 zijn controle netten -> deze zijn telkens afgeleid van combinatorische netten:
## neem ze weg bij netlijst generatie en ent ze met een dffeas op een willekeurig gegenereerd net om ze terug te plaatsen.